In [1]:
import geopandas as gpd
import pandas as pd
from shapely import wkt

In [2]:
df = pd.read_csv("jyvaskyla_routes_2024-05-23.csv")
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:3067")

gdf = gdf.to_crs("EPSG:4326")
gdf = gdf[gdf.geometry.type == 'LineString']

In [3]:
from shapely.geometry import Point

def determine_value(row):
    if row['1000m'] and not row['500m'] and not row['100m']:
        return 1
    elif row['1000m'] and row['500m'] and not row['100m']:
        return 2
    elif row['1000m'] and row['500m'] and row['100m']:
        return 3
    else:
        return 0
    
gdf['value'] = gdf.apply(determine_value, axis=1)

point = Point(25.74802, 62.2382)

new_point_gdf = gpd.GeoDataFrame({'geometry': [point]}, crs=gdf.crs)

frames = [gdf, new_point_gdf]

updated_gdf = pd.concat(frames, ignore_index=True)

In [4]:
updated_gdf.explore(column="value",cmap="viridis",marker_type="circle_marker", marker_kwds=dict(radius=10), style_kwds=dict(weight=2), tiles="CartoDB positron")